<a href="https://colab.research.google.com/github/kikiru328/Bone_Detection/blob/main/%EC%A2%85%EA%B5%AC/jongu_TJ_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""tjnet_model.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/18OMxML2I2nSKR-J8Zifw8IXiemsNM_mw
"""

import numpy as np
import tensorflow as tf
from tensorflow import keras

import torch

from keras.models import load_model
model = load_model('/content/drive/MyDrive/tjnet_model.h5')

bones = np.load('/content/drive/MyDrive/xdata.npy',allow_pickle=True) # .npy == 배열 파일
age = np.load('/content/drive/MyDrive/ydata.npy',allow_pickle=True)

#separable 합성곱 함수
def separable_conv(x,inchannel,outchannel):
  x = keras.layers.Conv2D(inchannel,(3,3),strides=1,padding="same")(x) # x축과 y축의 시작, 끝지점까지 하나하나의 픽셀을 검사하며 각각의 픽셀에 컨벌루션곱을 실행하며 2차원 배열 convout[i][j]에 이 정보를 저장하여 텐서 형식으로 아웃풋을 낸다.
  x = keras.layers.BatchNormalization()(x) # 배치 정규화(이전 레이어에서 결과량이 너무 커지거나 너무 작이지면 학습을 효과적으로 하지못하고 error rate가 수렴해버림을 방지)
  x = keras.layers.Conv2D(outchannel,(1,1),strides=1,padding="same")(x) # padding = 경계처리 방법(same = 입력사이즈와 출력사이즈가 동일), channel = 입력, 출력채널(컬러=3,흑백=1), stride = 지정값만큼 필터가 이동한다. default = 1
  x = keras.layers.BatchNormalization()(x)
  return x

#model middle_flow 함수
def middle_flow(input_x):
  #encoder
  x = keras.layers.MaxPool2D((2,2),padding="same")(input_x) # 컨벌루션 출력이미지의 주요 값만 추출해 작은 이미지를 만든다.(2,2) = 원본의 절반크기
  x = resiual_units(x) 
  x = keras.layers.MaxPool2D((2,2),padding="same")(x)
  x = resiual_units(x)
  x = keras.layers.MaxPool2D((2,2),padding="same")(x)
  x = resiual_units(x)
  
  

  #decoder
  x = resiual_units(x)
  x = keras.layers.UpSampling2D((2,2),interpolation='bilinear')(x)
  x = resiual_units(x)
  x = keras.layers.UpSampling2D((2,2),interpolation='bilinear')(x)
  x = resiual_units(x)
  x = keras.layers.UpSampling2D((2,2),interpolation='bilinear')(x)
  
  x = separable_conv(x,x.shape[-1],512)
  x = separable_conv(x,x.shape[-1],512) 
  
  #sigmoid 
  x = keras.activations.sigmoid(x)
  x = keras.layers.Multiply()([input_x,x])
  x = keras.layers.Add()([input_x,x])

  x = resiual_units(x)

  return x

#resiual_units 함수 
def resiual_units(input_x): 
  x = keras.layers.ReLU()(input_x) # 입력값이 0보다 작으면 0, 0보다 크면 입력값을 출력한다.
  x = separable_conv(x,x.shape[-1],128) # 입력이미지에 컨벌루션 곱을 행한다.
  x = keras.layers.BatchNormalization()(x)

  x = keras.layers.ReLU()(x)
  x = separable_conv(x,x.shape[-1],256)
  x = keras.layers.BatchNormalization()(x)

  x = keras.layers.ReLU()(x)
  x = separable_conv(x,x.shape[-1],512)
  x = keras.layers.BatchNormalization()(x)
  
  input_x = keras.layers.Add()([x,input_x])

  return input_x

#골연령 측정 모델
#entry flow model
input = keras.Input(shape=(256,256,5)) # 모델 정의
x = keras.layers.Conv2D(32,(3,3),strides=2)(input) #3X3 크기의 32층짜리 합성곱 신경망
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)

x = keras.layers.Conv2D(64,(3,3),strides=1)(x)
x = keras.layers.BatchNormalization()(x)
x = keras.layers.ReLU()(x)
#첫번째
x1 = keras.layers.Conv2D(128,(1,1),strides=2)(x) 
x1 = keras.layers.BatchNormalization()(x1)

x = separable_conv(x,x.shape[-1],128)
x = keras.layers.ReLU()(x)

x = separable_conv(x,x.shape[-1],128)
x = keras.layers.ReLU()(x)
x = keras.layers.MaxPool2D((2,2),2,padding="same")(x)

x = keras.layers.Add()([x,x1])
#2번째
x1 = keras.layers.Conv2D(512,(1,1),strides=2)(x)
x1 = keras.layers.BatchNormalization()(x1)

x = separable_conv(x,x.shape[-1],512)
x = keras.layers.ReLU()(x)

x = separable_conv(x,x.shape[-1],512)
x = keras.layers.ReLU()(x)
x = keras.layers.MaxPool2D((2,2),2,padding="same")(x)

x = keras.layers.Add()([x,x1])


#middle flow model
x = middle_flow(x)


#exit flow model
x1 = keras.layers.Conv2D(1024,(1,1),strides=2)(x)

x = keras.layers.ReLU()(x)
x = separable_conv(x,x.shape[-1],728)
x = keras.layers.ReLU()(x)
x = separable_conv(x,x.shape[-1],1024)
x = keras.layers.MaxPool2D((2,2),2,padding="same")(x)

x = keras.layers.Add()([x,x1])


x = separable_conv(x,x.shape[-1],1536)
x = keras.layers.ReLU()(x)
x = separable_conv(x,x.shape[-1],2048)
x = keras.layers.ReLU()(x)

x = keras.layers.GlobalAvgPool2D()(x) # 전역 평군 폴링층 = 각 맵의 특성의 평균을 출력 (하나의 맵에 하나의 값을 출력), 대부분의 값 소실, 출력층에서 유의미

x = keras.layers.Dense(1000,activation='relu')(x)
x = keras.layers.Dense(256,activation='relu')(x)
x = keras.layers.Dense(1)(x)

model = keras.models.Model(input,x)
model.compile(optimizer='adam',loss='mae',metrics=['mae','mse']) # adam = RMSProp + Momentum(방향과 스텝사이즈 적절), RMSProp = 이전맥락을 참조하여 보폭을 줄인다.
#https://onevision.tistory.com/entry/Optimizer-%EC%9D%98-%EC%A2%85%EB%A5%98%EC%99%80-%ED%8A%B9%EC%84%B1-Momentum-RMSProp-Adam
# mae = 에러 절대값의 평균, mse = 에러 제곱의 평균, binary_crossentropy = 결과값이 0과 1일경우 예측값이 둘 중 가까운 쪽으로 분류함 이진분류에 유의미,
#categorical crossentropy = 멀티클래스 분류에 사용 라벨이 원핫일 경우 사용, sparse categorical crossentropy = 멀티클래스 분류에 사용 라벨이 정수형태일 경우 사용

model.save('./tjnet_model.h5')

from tensorflow.python.keras.utils.vis_utils import model_to_dot
from IPython.display import SVG
import pydot
import graphviz

SVG(model_to_dot(model, show_shapes=True, show_layer_names=True, rankdir='TB',expand_nested=False, dpi=60, subgraph=False).create(prog='dot',format='svg'))

# Commented out IPython magic to ensure Python compatibility.
# %tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint

bones = np.load('/content/drive/MyDrive/xdata.npy',allow_pickle=True)
age = np.load('/content/drive/MyDrive/ydata.npy',allow_pickle=True)
X_data = bones[:,1]
y = age


filename = 'checkpoint-50-epochs-16-batchs.h5'
checkpoint = ModelCheckpoint(filename,mointor='val_loss',verbose=1,save_best_only=True,mode='auto')

from google.colab import drive
drive.mount('/content/drive')

tmp = np.zeros((256,256))
for _,x in enumerate(X_data):
  if _ == 0:
    tmp = x
    tmp = tmp.reshape(256,256,5)
    print(tmp)
  else:
    tmp = np.concatenate([tmp,x.reshape(256,256,5)])
    
X_data = tmp

np.set_printoptions(formatter={'float_kind': lambda x: "{0:0.1f}".format(x)})
y = y.astype(np.float)

train_x,test_x,train_y,test_y = train_test_split(X_data.reshape(1200,256,256,-1),y,random_state=42,test_size=0.2)
train_x,val_x,train_y,val_y = train_test_split(train_x,train_y,random_state=42,test_size=0.2)

with tf.device('/device:GPU:0'):
  model.fit(train_x,train_y,batch_size=16,epochs=150,callbacks=checkpoint,validation_data=(val_x,val_y))

prediction = model.predict(bones[400,1].reshape(-1,256,256,5))

output = round(prediction[0][0], 1)
print(output)

y[400]

model.summary()
